In [1]:
import pandas as pd
import numpy as np

# pd.set_option('display.max_colwidth',1000)

In [2]:
ht_df = pd.read_csv('./csv/sponsored/samples/hashtag_sponsored_2021-06-08 16_58_04.csv')
user_vids_df = pd.read_csv('./csv/sponsored/samples/users_videos_sponsored.csv')
user_dets_df = pd.read_csv('./csv/sponsored/samples/get_user_details_sponsored_2021-06-08 22_46_32.csv')

In [3]:
# Convert epoch times to datetime

user_vids_df['creation_time'] = pd.to_datetime(user_vids_df['creation_time'],unit='s')
user_dets_df['account_created'] = pd.to_datetime(user_dets_df['account_created'],unit='s')

In [4]:
# Calculate engagement metric for each video

user_vids_df['video_engagement'] = ((user_vids_df.diggs + 
                                     user_vids_df.comments + 
                                     user_vids_df.shares) / 
                                    user_vids_df.plays)

In [5]:
user_vids_df.columns

Index(['video_id', 'description', 'creation_time', 'duration', 'author_id',
       'username', 'nickname', 'music_id', 'song_title', 'music_author_name',
       'diggs', 'shares', 'comments', 'plays', 'video_engagement'],
      dtype='object')

In [6]:
user_vids_df['sponsored_ht'] = np.where(user_vids_df['description'].str.contains('#Sponsored|#sponsored|#ad |#Ad ', na=False), 1, 0)

In [7]:
user_vids_df['sponsored_ht'].value_counts(normalize=True)

0    0.98104
1    0.01896
Name: sponsored_ht, dtype: float64

In [8]:
user_vids_df['creation_time'].max()

Timestamp('2021-06-10 01:00:20')

In [9]:
user_vids_df[user_vids_df['username'] == 'paigemackenzie']['creation_time'].nlargest(20)

104979   2021-06-09 19:26:36
104980   2021-06-09 18:29:24
104981   2021-06-08 19:47:24
104982   2021-06-07 20:24:06
104983   2021-06-06 17:29:25
104984   2021-06-05 20:35:56
104985   2021-06-04 20:14:29
104986   2021-06-04 19:04:59
104987   2021-06-03 21:22:43
104988   2021-06-03 21:05:34
104989   2021-06-02 21:37:36
104990   2021-06-02 18:24:15
104991   2021-06-01 21:34:45
104992   2021-06-01 19:07:38
104993   2021-05-31 18:05:36
104994   2021-05-30 17:08:36
104995   2021-05-29 18:41:19
104996   2021-05-29 17:43:37
104997   2021-05-28 19:28:46
104998   2021-05-27 19:18:05
Name: creation_time, dtype: datetime64[ns]

In [10]:
recent_engagement = (user_vids_df.groupby('username').apply(lambda x: x.sort_values('creation_time', ascending=False).head(20)['diggs'].sum()) +
user_vids_df.groupby('username').apply(lambda x: x.sort_values('creation_time', ascending=False).head(20)['comments'].sum()) +
user_vids_df.groupby('username').apply(lambda x: x.sort_values('creation_time', ascending=False).head(20)['shares'].sum())) / user_vids_df.groupby('username').apply(lambda x: x.sort_values('creation_time', ascending=False).head(20)['plays'].sum())

In [12]:
type(recent_engagement)

pandas.core.series.Series

In [16]:
recent_engagement

username
208skindoc          0.119513
accelbyzantine      0.176226
adampukeonhaters    0.121376
adamselihi          0.066308
alexcortex          0.199094
                      ...   
yoeslan             0.063438
yoga.classes        0.063256
yun_bao             0.203785
zacharyyryann       0.122851
zhangarang          0.152309
Length: 193, dtype: float64

In [11]:
# user_dets_df['recent_engagement'] = 